In [ ]:
# pip install earthengine-api

In [ ]:
import ee
from datetime import datetime

# Authenticate to Earth Engine
ee.Authenticate()
ee.Initialize()

# Define your region of interest (ROI) using a GeoJSON geometry
roi = ee.Geometry.Polygon(
    [[[lon1, lat1], [lon2, lat2], [lon3, lat3], [lon4, lat4]]],
    None, False
)

# Define time range
start_date = '2021-01-01'
end_date = '2021-12-31'

# Create a Sentinel-1 image collection
sar_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(roi)
                  .filterDate(ee.Date(start_date), ee.Date(end_date))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                  .select(['VV', 'VH']))  # Select the polarization bands you need

# Function to mask clouds from Sentinel-1 images
def mask_s1_clouds(image):
    # Use the QA band to mask out cloudy pixels
    cloud_mask = image.select(['QA60']).bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloud_mask)

# Map the mask function over the collection
sar_collection = sar_collection.map(mask_s1_clouds)

# Get a list of image IDs
image_ids = sar_collection.aggregate_array('system:id').getInfo()

# Download each image
for image_id in image_ids:
    image = ee.Image(image_id)
    date = datetime.utcfromtimestamp(image.getInfo()['properties']['system:time_start'] / 1000.0)
    date_str = date.strftime('%Y-%m-%d')

    # Download the image
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'S1_{date_str}',
        folder='your_output_folder',  # Change to your desired output folder
        scale=10,  # Adjust the scale as needed
        region=roi
    )
    task.start()

    # Print status
    print(f'Downloading image for {date_str}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class VAE(nn.Module):
    def __init__(self, image_dim, rainfall_dim, latent_dim):
        super(VAE, self).__init__()

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(image_dim + rainfall_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim * 2)  # Two outputs for mean and log-variance
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + rainfall_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, image_dim),
            nn.Sigmoid()  # Assuming images are normalized between 0 and 1
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, image, rainfall):
        # Concatenate image and rainfall data for both encoder and decoder
        x = torch.cat([image, rainfall], dim=1)

        # Encoder
        encoder_output = self.encoder(x)
        mu, log_var = encoder_output[:, :latent_dim], encoder_output[:, latent_dim:]

        # Reparameterization trick
        z = self.reparameterize(mu, log_var)

        # Decoder
        decoder_input = torch.cat([z, rainfall], dim=1)
        reconstructed_image = self.decoder(decoder_input)

        return reconstructed_image, mu, log_var


In [ ]:
# Inside your training loop
num_epochs = 5
for epoch in range(num_epochs):
    for data in data_loader:
        # Extract satellite images and rainfall data
        images, rainfall = data

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        recon_images, mu, log_var = vae(images, rainfall)

        # Compute loss
        loss = criterion(recon_images, images, mu, log_var)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
